In [6]:
import pandas as pd
from textblob import TextBlob


In [7]:

# Specify the path to the CSV file
data_path = 'Womens Clothing E-Commerce Reviews.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(data_path)


In [8]:

def textblob_analysis(reviews):

    # Initialize empty lists
    polarities = []
    subjectivities = []
    
    # Iterate through reviews
    for review in reviews:
        
        # Create TextBlob 
        blob = TextBlob(review)
        
        # Conduct analysis
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        # Append scores
        polarities.append(polarity)
        subjectivities.append(subjectivity)

    # Return lists of scores     
    return polarities, subjectivities


In [9]:

# Sample dataframe
reviews = pd.Series(["Good product but it fades quickly.", "Bad service", "Okay I guess","""I do not like this item,
                      but my friend thinks it looks good on me, so I shall keep it."""]) 

# Run analysis
polarity_scores, subjectivity_scores = textblob_analysis(reviews)

# View results
print("The polarity of the reviews are:",polarity_scores)
print("The subjectivity of the reviews are:",subjectivity_scores)

The polarity of the reviews are: [0.5166666666666666, -0.6999999999999998, 0.5, 0.7]
The subjectivity of the reviews are: [0.55, 0.6666666666666666, 0.5, 0.6000000000000001]
